# Segment 4 Lab 5

Introducing a critical agent - the agent that brings it all together.

# Planning Agent

There are a number of frameworks out there that support building Agentic Workflows.

OpenAI has Swarm, LangChain has LangGraph, Gradio and HuggingFace have offerings, and there's Autogen from Microsoft, Crew.ai and many others.  

Each of these are abstractions on top of APIs to LLMs; some are lightweight, others come with significant functionality.

It's also perfectly possible - and sometimes considerably easier - to build an agentic solution by calling LLMs directly.

There's been considerable convergence on LLM APIs, and it's not clear that there's a need to sign up for one of the agent ecosystems for many use cases.

Anthropic has an [insightful post](https://www.anthropic.com/research/building-effective-agents) on building effective Agentic architectures that's well worth a read.

## Using Tools to give our Agent autonomy

In our case, we're going to create an Agent that uses Tools to make decisions about what to do next.

This is a bit over the top for this simple example, because we know exactly what the Agent is supposed to do. But it allows us to give the Agent some freedom..

Let's see how it works:

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()


OpenAI API Key exists and begins sk-proj-


In [3]:
# Use the Scanner agent from before

from agents.scanner_agent import ScannerAgent
scanner = ScannerAgent()

# Our tools

The next 3 cells have 3 **fake** functions that we will allow our LLM to call

In [4]:
def scan_the_internet_for_bargains():
    print(f"Scanning the internet")
    results = scanner.test_scan()
    return results.model_dump()

In [5]:
def estimate_true_value(description: str) -> str:
    print(f"Estimating true value of {description[:20]}...")
    return {"description": description, "estimated_true_value": 300}

In [6]:
def notify_user_of_deal(description: str, deal_price: float, estimated_true_value: float):
    print(f"Notifying user of {description} which costs {deal_price} and estimate is {estimated_true_value}")
    return {"notification_sent": "ok"}

## Telling the LLM about the tools it can use, with JSON

"Tool calling" is giving an LLM the power to run code on your computer!

Sounds a bit spooky?

The way it works is a little more mundane. We give the LLM a description of each Tool and the parameters, and we let it inform us if it wants any tool to be run.

It's not like OpenAI reaches in and runs a function. In the end, we have an if statement that calls our function if the model requests it.

In [7]:
scan_function = {
    "name": "scan_the_internet_for_bargains",
    "description": "Returns top bargains scraped from the internet along with the price each item is being offered for",
    "parameters": {
        "type": "object",
        "properties": {},
        "required": [],
        "additionalProperties": False
    }
}

In [8]:
estimate_function = {
    "name": "estimate_true_value",
    "description": "Given the description of an item, estimate how much it is actually worth",
    "parameters": {
        "type": "object",
        "properties": {
            "description": {
                "type": "string",
                "description": "The description of the item to be estimated"
            },
        },
        "required": ["description"],
        "additionalProperties": False
    }
}

In [9]:
notify_function = {
    "name": "notify_user_of_deal",
    "description": "Send the user a push notification about the most compelling deal",
    "parameters": {
        "type": "object",
        "properties": {
            "description": {
                "type": "string",
                "description": "The description of the item"
            },
            "deal_price": {
                "type": "number",
                "description": "The price offered by this deal scraped from the internet"
            }
            ,
            "estimated_true_value": {
                "type": "number",
                "description": "The estimated actual value that this is worth"
            }
        },
        "required": ["description", "deal_price", "estimated_true_value"],
        "additionalProperties": False
    }
}

In [10]:
tools = [{"type": "function", "function": scan_function},
        {"type": "function", "function": estimate_function},
        {"type": "function", "function": notify_function}]

## And now to bring it together - Tool calling in action

In [11]:
mapping = {"scan_the_internet_for_bargains": scan_the_internet_for_bargains, "estimate_true_value": estimate_true_value, "notify_user_of_deal": notify_user_of_deal}

def handle_tool_call(message):
    results = []
    for tool_call in message.tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = mapping.get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [12]:
system_message = "You are an Autonomous AI Agent that makes use of tools to carry out your mission. Your mission is to find great deals on bargain products, and notify the user when you find them."
user_message = "Your mission is to discover great deals on products. First you should scan the internet for bargain deals. Then for each deal, you should estimate its true value - how much it's actually worth. "
user_message += "Finally, you should pick the single most compelling deal where the deal price is much lower than the estimated true value, and send the user a push notification about that deal. "
user_message += "You must only notify the user about one deal, and be sure to pick the most compelling deal, where the deal price is much lower than the estimated true value. Then just respond OK to indicate success."
messages = [{"role": "system", "content": system_message},{"role": "user", "content": user_message}]

In [13]:
# A loop that repeatedly calls gpt-4o-mini until it has no more tools to call

done = False
while not done:
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        results = handle_tool_call(message)
        messages.append(message)
        messages.extend(results)
    else:
        done = True

print(response.choices[0].message.content)

Tool called: scan_the_internet_for_bargains
Scanning the internet
Tool called: estimate_true_value
Estimating true value of The Hisense R6 Serie...
Tool called: estimate_true_value
Estimating true value of The Poly Studio P21 ...
Tool called: estimate_true_value
Estimating true value of The Lenovo IdeaPad S...
Tool called: estimate_true_value
Estimating true value of The Dell G15 gaming ...
Tool called: estimate_true_value
Estimating true value of The certified refurb...
Tool called: notify_user_of_deal
Notifying user of The Hisense R6 Series 55R6030N is a 55-inch 4K UHD Roku Smart TV that offers stunning picture quality with 3840x2160 resolution. It features Dolby Vision HDR and HDR10 compatibility, ensuring a vibrant and dynamic viewing experience. The TV runs on Roku's operating system, allowing easy access to streaming services and voice control compatibility with Google Assistant and Alexa. With three HDMI ports available, connecting multiple devices is simple and efficient. which

## And now - putting all of this into a Planning Agent

In [14]:
from agents.autonomous_planning_agent import AutonomousPlanningAgent

In [15]:
import logging
root = logging.getLogger()
root.setLevel(logging.INFO)

In [16]:
import chromadb
DB = "products_vectorstore"
client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection('products')

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [17]:
agent = AutonomousPlanningAgent(collection)

INFO:root:[Autonomous Planning Agent] Autonomous Planning Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is ready
INFO:root:[Ensemble Agent] Initializing Ensemble Agent
INFO:root:[Specialist Agent] Specialist Agent is initializing - connecting to modal
INFO:root:[Specialist Agent] Specialist Agent is ready
INFO:root:[Frontier Agent] Initializing Frontier Agent
INFO:root:[Frontier Agent] Frontier Agent is set up with DeepSeek-R1 via Groq
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:root:[Frontier Agent] Frontier Agent is ready
INFO:root:[Random Forest Agent] Random Forest Agent is initializing
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-t

In [18]:
result = agent.plan()

INFO:root:[Autonomous Planning Agent] Autonomous Planning Agent is kicking off a run
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is calling scanner
INFO:root:[Scanner Agent] Scanner Agent is about to fetch deals from RSS feed
INFO:root:[Scanner Agent] Scanner Agent received 15 deals not already scraped
INFO:root:[Scanner Agent] Scanner Agent is calling OpenAI using Structured Output
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Scanner Agent] Scanner Agent received 5 selected deals with price>0 from OpenAI
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is estimating value via Ensemble Agent
INFO:root:[Ensemble Agent] Running Ensemble Agent - collaborating with specialist, frontier and random forest agents
INFO:root:

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products
INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call deepseek-r1-distill-llama-70b with RAG context of 5 similar products
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $39.99
INFO:root:[Random Forest Agent] Random Forest Agent is starting a prediction


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Random Forest Agent] Random Forest Agent completed - predicting $174.65
INFO:root:[Ensemble Agent] Ensemble Agent complete - returning $48.38
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is estimating value via Ensemble Agent
INFO:root:[Ensemble Agent] Running Ensemble Agent - collaborating with specialist, frontier and random forest agents
INFO:root:[Specialist Agent] Specialist Agent is calling remote fine-tuned model
INFO:root:[Specialist Agent] Specialist Agent completed - predicting $90.00
INFO:root:[Frontier Agent] Frontier Agent is using Llama 3.2 to preprocess the description
INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent is vectorizing using all-MiniLM-L6-v2


...

INFO:root:[Random Forest Agent] Random Forest Agent completed - predicting $211.54
INFO:root:[Ensemble Agent] Ensemble Agent complete - returning $123.41
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is estimating value via Ensemble Agent
INFO:root:[Ensemble Agent] Running Ensemble Agent - collaborating with specialist, frontier and random forest agents
INFO:root:[Specialist Agent] Specialist Agent is calling remote fine-tuned model
INFO:root:[Specialist Agent] Specialist Agent completed - predicting $950.00
INFO:root:[Frontier Agent] Frontier Agent is using Llama 3.2 to preprocess the description
INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent is vectorizing using all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products
INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call deepseek-r1-distill-llama-70b with RAG context of 5 similar products
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $1099.99
INFO:root:[Random Forest Agent] Random Forest Agent is starting a prediction


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Random Forest Agent] Random Forest Agent completed - predicting $376.65
INFO:root:[Ensemble Agent] Ensemble Agent complete - returning $1015.33
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is notifying user
INFO:root:[Messaging Agent] Messaging Agent is using Claude to craft the message
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:root:[Messaging Agent] Messaging Agent is sending a push notification
INFO:root:[Messaging Agent] Messaging Agent has completed
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Autonomous Planning Agent] Autonomous Planning Agent completed with: OK


# Finally - with a Gradio UI

In [19]:
# Reset memory back to 2 deals discovered in the past

from deal_agent_framework import DealAgentFramework
DealAgentFramework.reset_memory()

In [ ]:
!python price_is_right.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
[2025-03-04 22:22:46 -0500] [Agents] [INFO] [Agent Framework] Initializing Agent Framework
[2025-03-04 22:22:46 -0500] [Agents] [INFO] [Agent Framework] Initializing Agent Framework
[2025-03-04 22:22:46 -0500] [Agents] [INFO] [Autonomous Planning Agent] Autonomous Planning Agent is initializing
[2025-03-04 22:22:46 -0500] [Agents] [INFO] [Autonomous Planning Agent] Autonomous Planning Agent is initializing
[2025-03-04 22:22:46 -0500] [Agents] [INFO] [Scanner Agent] Scanner Agent is initializing
[2025-03-04 22:22:46 -0500] [Agents] [INFO] [Scanner Agent] Scanner Agent is initializing
[2025-03-04 22:22:46 -0500] [Agents] [INFO] [Scanner Agent] Scanner Agent is ready
[2025-03-04 22:22:46 -0500] [Agents] [INFO] [Scanner Agent] Scanner Agent is ready
[2025-03-04 22:22:46 -0500] [Agents] [INFO] [Ensemble Agent] Initializing Ensemble Agent
[2025-03-04 22:22:46 -0500] [Agents] [INFO] [Ensem